# High- and Low-tide Composites

This notebook combines some of the lessons learnt in the first notebook
and applies them to an analysis that seeks to create cloud-free mosaic
images at either high or low tide. This can be helpful if you need to
run an analysis such as identifying seagrass, which is more difficult
when it's covered by water.

### Package import

First, we configure our Python package imports

In [1]:
from pystac_client import Client
from odc.stac import load
from dea_tools.coastal import pixel_tides

from dask.distributed import Client as DaskClient

from pathlib import Path
import folium

### Configuration

We're using the Element-84 STAC API to find Sentinel-2 data. This cell
also sets up a Dask client, so that we can lazy-load data and process
efficiently across multiple threads.

If you click the link to the Dashboard, you should be able to watch Dask
work, once we're at the stage where we compute the results.

In [ ]:
# STAC Catalog URL
catalog = "https://earth-search.aws.element84.com/v1"

# Create a STAC Client
client = Client.open(catalog)

# Set up Dask
dask_client = DaskClient(n_workers=2, threads_per_worker=16)
dask_client

### Location and time

This step sets up the spatial and temporal extents as well as configuring
where the tide model is stored.

In [3]:
# Find a location you're interested in on Google Maps and copy the coordinates
# by right-clicking on the map and clicking the coordinates

# These coords are in the order Y then X, or Latitude then Longitude
# clear_image = 2

coords = 9.9251702,118.7817471,13.79  # Honda Bay, Palawan, Philippines
clear_image = 0

buffer = 0.1
bbox = (coords[1] - buffer, coords[0] - buffer, coords[1] + buffer, coords[0] + buffer)

datetime = "2023/2024"
s2_stretch = {
    "vmin": 1000,
    "vmax": 4000,
}


# Tide data and config
home = Path("~")
tide_data_location = f"{home}/tide_models"

## Find and load data

First, search the STAC API for Sentinel-2 scenes over our study site

In [ ]:
items = client.search(
    collections=["sentinel-2-c1-l2a"],
    bbox=bbox,
    datetime=datetime
).item_collection()

print(f"Found {len(items)} STAC items")

### Load

Next, we use `odc-stac` to load data.

In [ ]:
data = load(
    items,
    bands=["red", "green", "blue", "swir16", "cloud", "scl"],
    bbox=bbox,
    groupby="solar_day",
    chunks={"x": 2048, "y": 2048},
)

data

### Cloud masking

Masking out clouds is important, so that we can make a cloud-free mosaic
using as many pixels as possible. In the previous example, we selected
Landsat scenes that had less than a certain percentage of clouds. This method
is good, but potentially loses valuable data by discarding the more cloudy
scenes.

In [6]:
# 1: defective, 3: shadow, 8: med confidence cloud, 9: high confidence cloud, 10: cirrus
mask_flags = [1, 3, 8, 9, 10]

cloud_mask = data.scl.isin(mask_flags)

masked = data.where(~cloud_mask).drop_vars("scl")

In [ ]:
# Let's compare before and after

# Before
data[["red", "green", "blue"]].isel(time=clear_image).to_array().plot.imshow(size=8, **s2_stretch)

In [ ]:
# After
masked[["red", "green", "blue"]].isel(time=clear_image).to_array().plot.imshow(size=8, **s2_stretch)

### Tide modelling

This next cell annotates the data with the height of the tide at
the time the scene was captured. We use this information to select
the scenes at the top and bottom 30% of tides.

In [ ]:
# Annotate the data with the tide height
tides_lowres = pixel_tides(
    masked, resample=False, directory=tide_data_location, model="FES2022", dask_compute=True
)

In [ ]:
lowest, highest = tides_lowres.quantile([0.3, 0.7]).values

low_scenes = tides_lowres.where(tides_lowres < lowest, drop=True)
high_scenes = tides_lowres.where(tides_lowres > highest, drop=True)

data_low = masked.sel(time=low_scenes.time)
data_high = masked.sel(time=high_scenes.time)

print(f"Found {len(data_low.time)} low tide days and {len(data_high.time)} high tide days out of {len(data.time)} days")

## Compute and visualise

This next cell creates a median of the high or low tide scenes. The
function call `compute()` at the end tells Dask to do the work of
downloading data and running the computation.

In [ ]:
# This will take up to 10 minutes to complete
median_low = data_low.median("time").compute()
median_high = data_high.median("time").compute()

### Visualisation

First we are going to visualise just the high-tide scene. If this looks
good, the next step is to create an index, which will highlight areas that
are likely to be land and water, and we can see the difference between
low and high tide clearly. Each visualisation is done using an interactive
map.

In [ ]:
median_high.odc.explore(**s2_stretch)

In [13]:
# Calculate MNDWI
median_low["mndwi"] = (median_low.green - median_low.swir16) / (median_low.green + median_low.swir16)
median_high["mndwi"] = (median_high.green - median_high.swir16) / (median_high.green + median_high.swir16)

In [ ]:
# Plot LOW and HIGH MNDWI on the same map
m = folium.Map()

# Add the visual RGB
median_low.odc.to_rgba(**s2_stretch).odc.add_to(m, name="RGB (low)")
median_high.odc.to_rgba(**s2_stretch).odc.add_to(m, name="RGB (high)")

arguments = {
    "cmap": "RdBu",
    "vmin": -0.5,
    "vmax": 0.5,
}

# Plot each sample image with different colormap
median_low.mndwi.odc.add_to(m, name="MNDWI (low)", **arguments)
median_high.mndwi.odc.add_to(m, name="MNDWI (high)", **arguments)

folium.LayerControl().add_to(m)
m.fit_bounds(median_low.odc.map_bounds())
m